In [31]:
import pandas as pd
import numpy as np
import sqlalchemy as sql
from dotenv import load_dotenv
import os
import re

In [3]:
load_dotenv()
user = os.getenv('DB_USERNAME')
db = os.getenv('DB')
host = os.getenv('DB_HOST')
password = os.getenv('DB_PASSWORD')

connection_string = f"mysql+pymysql://{user}:{password}@{host}/{db}?charset=utf8mb4"
engine = sql.create_engine(connection_string)


In [8]:
pd.set_option("display.max_columns", None)

In [ ]:
#one-hot encode cmc; colors; keywords; produced_mana; 
#average cost per power in set, trade in set, kill in set
#extract type;
#add trade
#process text (token; replace card name; number of words/char; TF-IDF)
#try to process creature type
#destroy permanent, target sacrifice, life gain, deal damage, loose, draw card (bool or number)

In [12]:
query = """SELECT d.name, scryfall_id, oracle_text, cmc, colors, keywords, type_line, set_name, `set`, d.rarity, `power`, toughness, produced_mana, loyalty, avg_pick FROM scryfall_cards s
INNER JOIN draft_data d ON d.scryfall_id = s.id
WHERE drat_format = 'PremierDraft';
"""

In [61]:
with engine.connect() as conn:
    df = pd.read_sql_query(query, conn)

In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5714 entries, 0 to 5713
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   scryfall_id    5714 non-null   object 
 1   oracle_text    5714 non-null   object 
 2   cmc            5714 non-null   object 
 3   colors         5714 non-null   object 
 4   keywords       5714 non-null   object 
 5   type_line      5714 non-null   object 
 6   set_name       5714 non-null   object 
 7   set            5714 non-null   object 
 8   rarity         5714 non-null   object 
 9   power          5714 non-null   object 
 10  toughness      5714 non-null   object 
 11  produced_mana  5714 non-null   object 
 12  loyalty        5714 non-null   object 
 13  avg_pick       5714 non-null   float64
dtypes: float64(1), object(13)
memory usage: 625.1+ KB


In [63]:
df["type_line"].head(10)

0    Legendary Creature — Human Rogue
1         Creature — Phyrexian Cleric
2                  Artifact — Vehicle
3               Creature — Orc Knight
4                 Artifact — Treasure
5           Creature — Human Elf Monk
6      Legendary Creature — Rat Pilot
7                             Sorcery
8     Legendary Creature — Frog Beast
9                         Enchantment
Name: type_line, dtype: object

In [64]:
df["card_type"] = df["type_line"].apply(lambda x: x.split(" — ")[0])

In [65]:
df["card_type"].head(10)

0    Legendary Creature
1              Creature
2              Artifact
3              Creature
4              Artifact
5              Creature
6    Legendary Creature
7               Sorcery
8    Legendary Creature
9           Enchantment
Name: card_type, dtype: object

In [80]:
df["Legenday"] = df["card_type"].apply(lambda x: "Legendary" in x)

In [81]:
df["card_type"] = df["card_type"].apply(lambda x: x.split(" ")[1] if "Legendary" in x else x)

In [66]:
df["power_*"] = df["power"].apply(lambda x: "*" in x)

In [67]:
df["toughness_*"] = df["toughness"].apply(lambda x: "*" in x)

In [68]:
df[["power_*", "toughness_*"]].head(10)

,power_*,toughness_*
0,False,False
1,False,False
2,False,False
3,False,False
4,False,False
5,False,False
6,False,False
7,False,False
8,True,False
9,False,False


In [69]:
def convert_power_tough(x):
    pattern = re.compile(r"\d+")
    if "*" in x:
        match = re.findall(pattern, x)
        if match == []:
            return int(0)
        else:
            return int(match[0])
    else:
        return int(x)

In [70]:
convert_power_tough("12")

12

In [71]:
df["power"] = df["power"].apply(convert_power_tough)

In [72]:
df["toughness"] = df["toughness"].apply(convert_power_tough)

In [73]:
df["power"].head(10)

0    3
1    4
2    4
3    4
4    0
5    1
6    4
7    0
8    0
9    0
Name: power, dtype: int64

In [77]:
df["power_toughness"] = df["power"] + df["toughness"]

In [78]:
df["power_toughness"].head(10)

0    4
1    8
2    7
3    8
4    0
5    5
6    7
7    0
8    5
9    0
Name: power_toughness, dtype: int64

In [85]:
df["cmc"] = df["cmc"].apply(float)

In [99]:
creature_df = df[df["card_type"] == "Creature"]


grouped_sum = creature_df.groupby("set").agg({"power_toughness": "sum", "cmc": "sum"})


average_stat_cost = grouped_sum["cmc"] / grouped_sum["power_toughness"]


df = df.merge(average_stat_cost.rename("average_stat_cost"), how="left", on="set")

In [100]:
df["card_average_stat_cost"] = df["cmc"] / df["power_toughness"]

In [106]:
#highly dubious
df["card_average_stat_cost"] = df["card_average_stat_cost"].apply(lambda x: 1 if x > 100 else x)

In [107]:
df[["scryfall_id", "set", "average_stat_cost", "card_average_stat_cost"]]

,scryfall_id,set,average_stat_cost,card_average_stat_cost
0,f01f12e0-f354-43aa-9e2d-b59a99571a5f,SNC,0.629010,0.750000
1,626c46a3-72b8-4e04-adf2-c9c7aaf94f04,ONE,0.566906,0.625000
2,28273a5b-57b3-4b7a-b017-5886c171c9c9,NEO,0.578534,0.428571
3,3620ed96-1d15-4942-b9e5-9f9a64b0cab4,AFR,0.598179,0.625000
4,2f375674-7ae2-4430-b1f3-c26fa5b201d1,SNC,0.629010,1.000000
...,...,...,...,...
5709,e8815cd9-7032-445a-aebc-cfc19bd51ee4,DOM,0.610132,0.600000
5710,519c5660-4daf-4404-b808-66d1c840ab70,XLN,0.651203,1.000000
5711,e3979b88-ac58-420a-8c03-37ea5d93d0f1,XLN,0.651203,1.000000
5712,b3815ab6-87cd-4310-8068-ec721ee10a24,RNA,0.634146,0.500000
